In [16]:
import pandas as pd
import numpy as np

In [17]:
df=pd.read_csv('bestsellers with categories.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


# Hypothesis Testing for Mean
* datanya Continuous
* **Parametrik**
* membandingkan **rata-rata** dari populasi
* One sample T-test, T-test paired, Two independent samples T-test, One way ANOVA/F test for more that 2 population mean
* Termasuk Uji Parametrik
    * memiliki asumsi2 tertentu yang perlu dipenuhi
    * asumsi distribusi normal

## One Sample T-test
membandingkan rata-rata pengukuran dari suatu kelompok (A) dengan nilai teoritis yang diharapkan.  

Di contoh kali ini kita akan coba membuktikan hipotesis bahwa rata-rata harga buku di Amazon adalah 10 atau di atas 10.

**Hypothesis**  
Ho: Rata-rata harga buku di Amazon bestseller adalah 10 usd (praduga tak bersalah)  
Ha: Rata-rata harga buku di Amazon lebih besar dari 10 usd (alternatif)  (1 tailed t test)

In [4]:
from scipy.stats import ttest_1samp

ttest_1samp(df['Price'],10) #(data, estimasi pembanding)
# perhatikan di hasil p-val, terdapat e-11, artinya p value kita memiliki nilai dikali 10 pangkat -11

Ttest_1sampResult(statistic=6.705376094208877, pvalue=5.001628960126041e-11)

In [5]:
ttest_1samp(df['Price'],10,alternative='greater') #kalau pakai alternatif, tidak perlu dibagi 2 lagi p valuenya

Ttest_1sampResult(statistic=6.705376094208877, pvalue=2.5008144800630206e-11)

In [8]:
# conclusion

tstats, tpval= ttest_1samp(df['Price'],10)
 
if tpval/2<=0.05:# Ha membandingkan secara 1 arah/ 1 tailed, jadi p val perlu dibagi 2 
    print('Tolak Ho, rata-rata harga >10 USD')
else:
    print('Terima Ho, rata-rata harga tidak lebih besar dari 10')

Tolak Ho, rata-rata harga >10 USD


Kesimpulan: P value lebih rendah dari tingkat signifikansi 0.05, artinya, kita punya cukup bukti untuk menolak Ho, jadi rata-rata buku lebih besar dari 10 USD, dengan tingkat signifikansi 5% atau tingkat kepercayaan 95%.

In [10]:
df['Price'].mean() #ketika di cek rata2 harga buku 13.1, dan ini dinilai signifikan lebih tinggi dari 10 oleh uji statistik

13.1

## Two Independent Samples T-test  
digunakan untuk membandingkan **rata-rata** dari 2 populasi A dan B yang independen/tidak saling berkaitan.  
Apakah rata-rata di populasi A berbeda dari rata-rata populasi B?  

Pada contoh berikut, populasi A kita adalah buku Fiksi, sedangkan populasi B adalah buku non fiksi. Kita ingin membandingkan rata-rata harga buku di kedua populasi.  

* Asumsi: distribusi harga buku normal

In [13]:
df[['Genre','Price']].head()

,Genre,Price
0,Non Fiction,8
1,Fiction,22
2,Non Fiction,15
3,Fiction,6
4,Non Fiction,12


**Hypotesis**    
Ho: Rata-rata harga buku Fiksi = rata-rata harga buku non Fiksi  
Ha: Rata-rata haarga buku Fiksi >< rata-rata harga buku non Fiksi (two tailed test)  

In [14]:
# uji 1 tailed biasanya ditandai dengan >, < pada Ha
# uji 2 tailed ditandai dengan tanda tidak sama dengan pada Ha

Pada Two Independent Samples T-test, terdapat 2 uji, tergantung dari nilai variance kedua populasi.    

Kalau nilai variance kedua populasi tidak seimbang (salah satu memiliki nilai lebih dari 2 kali lipat yang lainnya), gunakan Non equal Variance assumption.  
Kalau nilai variance seimbang, maka gunakan equal variance assumption.

### Non Equal Variance Assumption

In [16]:
df[df['Genre']=='Fiction']['Price'] # harga buku fiksi

1      22
3       6
5      11
6      30
7      15
       ..
541     9
542     9
543     9
544     9
545     8
Name: Price, Length: 240, dtype: int64

In [32]:
from scipy.stats import ttest_ind

ttest_ind(
    df[df['Genre']=='Fiction']['Price'],
    df[df['Genre']=='Non Fiction']['Price'],
    equal_var=False #karena non equal variance
)

Ttest_indResult(statistic=-4.537208093329263, pvalue=7.019009601772709e-06)

In [31]:
tstats, tpval=ttest_ind(
    df[df['Genre']=='Fiction']['Price'],
    df[df['Genre']=='Non Fiction']['Price'], 
    equal_var=False 
)

if tpval<=0.05: #2tailed, tidak perlu dibagi 2
    print('Tolak Ho, rata-rata buku tidak fiksi sama dengan non fiksi')
else:
    print('Terima Ho, rata-rata buku fiksi sama dengan non fiksi')

Tolak Ho, rata-rata buku tidak fiksi sama dengan non fiksi


Kesimpulan: P val lebih rendah dari tingkat signifikasi (alpha) 0.05, artinya kita memiliki cukup bukti untuk menolak Ho, jadi, harga buku fiksi tidak sama dengan buku non fiksi.

Coba kita cek manual harga buku dari masing-masing genre.

In [22]:
df[['Genre','Price']].groupby('Genre').mean()
# beda 4 usd, kita bisa coba uji ulang, dengan Ha harga buku non fiksi > dari ahrga buku fiksi

,Price
Genre,
Fiction,10.850000
Non Fiction,14.841935


### Equal Variance Assumption

In [23]:
df[df['Genre']=='Fiction']['Price'].var()

73.49205020920503

In [24]:
df[df['Genre']=='Non Fiction']['Price'].var()

145.0396596721994

In [26]:
df[df['Genre']=='Non Fiction']['Price'].var()/df[df['Genre']=='Fiction']['Price'].var()

#perbandingan variance tidak lebih besar dari 2, jadi seharusya kita pakai  asumsi equal variance

1.9735421621702545

In [28]:
tstats, tpval=ttest_ind(
    df[df['Genre']=='Fiction']['Price'],
    df[df['Genre']=='Non Fiction']['Price'],
    equal_var=True #karena asumsi equal variance
)
print(tstats,tpval)

if tpval<=0.05: #2tailed, tidak perlu dibagi 2
    print('Tolak Ho, rata-rata buku tidak fiksi sama dengan non fiksi')
else:
    print('Terima Ho, rata-rata buku fiksi sama dengan non fiksi')

-4.351608076818746 1.612435071812073e-05
Tolak Ho, rata-rata buku tidak fiksi sama dengan non fiksi
